Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import numpy as np
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

In [0]:
# Define a function to wrangle train, validate, and test sets in the same way. Clean outliers and engineer features.

def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""

    # Prevent SettingWithCopyWarning
    X = X.copy()

    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)

    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    # Also create a "missing indicator" column, because the fact that value
    # are missing be a predictive signal.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year',
                       'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()

    # Drop num_private because it has mostly zeros
    X = X.drop('num_private')

    # Drop duplicate columns
    duplicates = ['quantity_group', 'payment_type']
    X = X.drop(columns=duplicates)

    # Drop recorded_by (never varies) and id (always varies, random)
    unusable_variance = ['recorded_by', 'id']
    X = X.drop(columns=unusable_variance)

    # Conver date recorded to datetime
    X['date_recoded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)

    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')

    # Engineer feature: how many years from contrution_year to date_recorded
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()

    # Engineer your own feature:
    # Difference in Latitude and Longitude from Tanzania's 10 biggest cities.
    #1
    X['longitude_difference_from_Dar_es_Salaam'] = X['longitude'] - 39.2083
    X['latitude_difference_from_Dar_es_Salaam'] = X['latitude'] - 6.7924
    #2
    X['longitude_difference_from_Mwanza'] = X['longitude'] - 32.9175
    X['latitude_difference_from_Mwanza'] = X['latitude'] - 2.5164
    #3
    X['longitude_difference_from_Arusha'] = X['longitude'] - 36.6830
    X['latitude_difference_from_Arusha'] = X['latitude'] - 3.3869
    #4
    X['longitude_difference_from_Dodoma'] = X['longitude'] - 35.7516
    X['latitude_difference_from_Dodoma'] = X['latitude'] - 6.1630
    #5    
    X['longitude_difference_from_Mbeya'] = X['longitude'] - 33.4608
    X['latitude_difference_from_Mbeya'] = X['latitude'] - 8.9094
    #6
    X['longitude_difference_from_Morogoro'] = X['longitude'] - 37.6591
    X['latitude_difference_from_Morogoro'] = X['latitude'] - 6.8278
    #7
    X['longitude_difference_from_Tanga'] = X['longitude'] - 39.1023
    X['latitude_difference_from_Tanga'] = X['latitude'] - 5.0889
    #8
    X['longitude_difference_from_Kahama'] = X['longitude'] - 32.5938
    X['latitude_difference_from_Kahama'] = X['latitude'] - 3.8376
    #9
    X['longitude_difference_from_Tabora'] = X['longitude'] - 32.8197
    X['latitude_difference_from_Tabora'] = X['latitude'] - 5.0425
    #10
    X['longitude_difference_from_Zanzibar City'] = X['longitude'] - 39.2026
    X['latitude_difference_from_Zanzibar City'] = X['latitude'] - 6.1659

    # return in wrangled dataframe
    return X

    train = wrangle(train)
    test = wrangle(test)

In [0]:
train

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,Pangani,Kiduruni,Kilimanjaro,3,5,Hai,Masama Magharibi,125,True,GeoData Consultants Ltd,Water Board,Losaa Kia water supply,True,1999,gravity,gravity,gravity,water board,user-group,pay per bucket,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,Rufiji,Igumbilo,Iringa,11,4,Njombe,Ikondo,56,True,GeoData Consultants Ltd,VWC,Ikondo electrical water sch,True,1996,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,Rufiji,Madungulu,Mbeya,12,7,Mbarali,Chimala,0,True,GeoData Consultants Ltd,VWC,NaN,False,0,swn 80,swn 80,handpump,vwc,user-group,pay monthly,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,Rufiji,Mwinyi,Dodoma,1,4,Chamwino,Mvumi Makulu,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional


In [0]:
# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists
features = numeric_features + categorical_features

In [0]:
# Arrange data into X features matrix and y target vector
X_train = train[features]
y_train = train[target]
#X_test = test[features]

In [0]:
# Use a model for classification

# Try XGBoost Classifier.
 
#If you’re doing a multi-class classification problem 
# — such as whether a waterpump is functional, functional needs repair, or nonfunctional — 
# then use a categorical encoding that works for multi-class classification, such as OrdinalEncoder)

# Try Ordinal Encoder

%%time

from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform


X_train = train.drop(columns=target)
y_train = train[target]

pipeline = make_pipeline(
    ce.ordinal.OrdinalEncoder(),
    SimpleImputer(),
    XGBClassifier(),
)

param_distributions = {
    #'targetencoder__min_samples_leaf': randint(1,1000),
    'simpleimputer__strategy': ['mean', 'median'],
    #Use hyperparameters that match the classifier
    'xgbclassifier__learning_rate': (0.01,0.2),
    'xgbclassifier__max_depth': (3,10),
    'xgbclassifier__min_child_weight': (0,1),
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=50,
    cv=5,
    # Use a metric for classification, such as accuracy
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 5 folds for each of 16 candidates, totalling 80 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 16 is smaller than n_iter=50. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   33.3s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed: 16.0min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 24.6min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 28.2min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed: 38.2min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 42.5min
[Parallel(n_jobs=-1)]: Done  80 

CPU times: user 1min 38s, sys: 529 ms, total: 1min 39s
Wall time: 54min 32s


In [0]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation Accuracy', search.best_score_)

Best hyperparameters {'xgbclassifier__min_child_weight': 1, 'xgbclassifier__max_depth': 10, 'xgbclassifier__learning_rate': 0.2, 'simpleimputer__strategy': 'mean'}
Cross-validation Accuracy 0.7995959595959596


In [0]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,7,15,6,14,2,10,3,11,5,13,4,12,0,1,8,9
mean_fit_time,113.916,114.358,117.796,118.163,129.997,129.397,129.565,129.433,30.1037,29.9649,29.982,29.9973,30.3182,30.0495,30.3188,30.048
std_fit_time,0.694183,0.841352,0.42237,0.611366,0.965519,0.57643,0.454936,0.565089,0.0873444,0.0671371,0.155357,0.114834,0.129062,0.156505,0.156226,0.0948175
mean_score_time,0.831786,0.88033,0.895523,0.908261,0.926786,0.915115,0.948692,0.922001,0.355269,0.35258,0.352544,0.348068,0.300019,0.301314,0.299618,0.34004
std_score_time,0.0471594,0.062616,0.0501786,0.0562651,0.0483428,0.0342936,0.0535643,0.035028,0.00702482,0.00774103,0.0119799,0.00853276,0.0053786,0.00790797,0.00772116,0.049515
param_xgbclassifier__min_child_weight,1,1,0,0,0,0,1,1,1,1,0,0,0,1,0,1
param_xgbclassifier__max_depth,10,10,10,10,10,10,10,10,3,3,3,3,3,3,3,3
param_xgbclassifier__learning_rate,0.2,0.2,0.2,0.2,0.01,0.01,0.01,0.01,0.2,0.2,0.2,0.2,0.01,0.01,0.01,0.01
param_simpleimputer__strategy,mean,median,mean,median,mean,median,mean,median,mean,median,mean,median,mean,mean,median,median
params,"{'xgbclassifier__min_child_weight': 1, 'xgbcla...","{'xgbclassifier__min_child_weight': 1, 'xgbcla...","{'xgbclassifier__min_child_weight': 0, 'xgbcla...","{'xgbclassifier__min_child_weight': 0, 'xgbcla...","{'xgbclassifier__min_child_weight': 0, 'xgbcla...","{'xgbclassifier__min_child_weight': 0, 'xgbcla...","{'xgbclassifier__min_child_weight': 1, 'xgbcla...","{'xgbclassifier__min_child_weight': 1, 'xgbcla...","{'xgbclassifier__min_child_weight': 1, 'xgbcla...","{'xgbclassifier__min_child_weight': 1, 'xgbcla...","{'xgbclassifier__min_child_weight': 0, 'xgbcla...","{'xgbclassifier__min_child_weight': 0, 'xgbcla...","{'xgbclassifier__min_child_weight': 0, 'xgbcla...","{'xgbclassifier__min_child_weight': 1, 'xgbcla...","{'xgbclassifier__min_child_weight': 0, 'xgbcla...","{'xgbclassifier__min_child_weight': 1, 'xgbcla..."
split0_test_score,0.790404,0.790404,0.790152,0.790152,0.787205,0.787205,0.788721,0.788721,0.756313,0.756313,0.758081,0.758081,0.716414,0.716414,0.716414,0.716414


In [0]:
pipeline = search.best_estimator_

In [0]:
# Use a metric for classification, such as accuracy
from sklearn.metrics import accuracy_score

X_test = test[features]
y_test = test[target]

y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: ${accuracy:}')
# This is a strange rare error and I don't want to submit this to the Kaggle challenge for a lower score anyways.

KeyError: ignored

In [0]:
import pandas as pd
 
# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']
 
target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]
 
sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)

In [0]:
# Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. 
# Use the blue Submit Predictions button to upload your CSV file. 
# Or you can use the Kaggle API to submit your predictions.)

# estimator is your model or pipeline, which you've fit on X_train

# X_test is your pandas dataframe or numpy array, 
# with the same number of rows, in the same order, as test_features.csv, 
# and the same number of columns, in the same order, as X_train

#y_pred = estimator.predict(X_test)
y_pred = pipeline.predict(X_test)

# Makes a dataframe with two columns, id and status_group, 
# and writes to a csv file, without the index

sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('John_Dailey_KaggleWaterPumps_OrdinalEnc_XGBoost_crossval.csv', index=False)

from google.colab import files
files.download('John_Dailey_KaggleWaterPumps_OrdinalEnc_XGBoost_crossval.csv')

In [0]:
# Commit your notebook to your fork of the GitHub repo.